In [51]:
import pandas as pd
import numpy as np
import networkx as nx
from bs4 import BeautifulSoup
import requests
import codecs
import os
from lxml import etree
import json
import unicodedata
import string

path = '/content/drive/MyDrive/ДИПЛОМ/Списки компаний/Парсеры для карточек компаний/'

In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [72]:
data = pd.read_excel('/content/drive/MyDrive/ДИПЛОМ/Списки компаний/Парсеры для карточек компаний/company_list_fst_iteration.xlsx', converters={'ИНН':str})
data['ИНН'] = data['ИНН'].astype(str)
data.drop(['№', 'Столбец1', 'Есть выгрузка', 'Место по итогам 2020 года',
           'Объем реализации в 2021 году (млн. рублей)*', 'Объем реализации в 2020 году (млн. рублей)*',
           'Темпы роста выручки за год (%)', 'Объем реализации в 2021 году (млн. долларов)', 'Прибыль до налогообложения в 2021 году (млн. рублей)',
           'Чистая прибыль в 2021 году (млн. рублей)', 'Тип отчетности'], axis=1, inplace=True)
data.rename(columns={'Название':'company_file','ИНН':'inn_file','Отрасль':'industry_file'}, inplace=True)
data

,company_file,inn_file,industry_file
0,"«КТЗ», промышленный оператор",9704082895,оптовая торговля
1,«Автомир-Трейд»,7730569353,розничная торговля
2,«Джи Пи Си Рус»,7703400140,платежный сервис
3,«Самотлорнефтепромхим»,8603118208,сервисные компании
4,«Мосметрострой»,7710068052,"инжиниринг, промышленно-инфраструктурное строи..."
...,...,...,...
583,X5 Group,7733571872,розничная торговля
584,РЖД,7708503727,транспорт и логистика
585,"«Роснефть», нефтяная компания",7706107510,нефтяная и нефтегазовая промышленность
586,«ЛУКОЙЛ»,7708004767,нефтяная и нефтегазовая промышленность


In [73]:
list_of_smi_publications = pd.DataFrame(os.listdir(path+'company_smi_publications'), dtype=str)
list_of_smi_publications.rename(columns={0:'inn_smi_publications'}, inplace=True)
list_of_smi_publications['inn_smi_publications'] = list_of_smi_publications['inn_smi_publications'].str.replace('.html', '', regex=False).astype(str)
list_of_smi_publications

,inn_smi_publications
0,9704082895
1,7730569353
2,7703400140
3,8603118208
4,7710068052
...,...
559,7708503727
560,7706107510
561,7708004767
562,7736050003


In [74]:
list_of_company_main_cards = pd.DataFrame(os.listdir(path+'company_main_cards'))
list_of_company_main_cards.rename(columns={0:'inn_company_main_cards'}, inplace=True)
list_of_company_main_cards['inn_company_main_cards'] = list_of_company_main_cards['inn_company_main_cards'].str.replace('.html', '', regex=False)
list_of_company_main_cards

,inn_company_main_cards
0,9704082895
1,7730569353
2,7703400140
3,8603118208
4,7710068052
...,...
574,7708503727
575,7706107510
576,7708004767
577,7736050003


In [75]:
all_inn_data = pd.merge(list_of_smi_publications, list_of_company_main_cards, how='inner', left_on='inn_smi_publications', right_on='inn_company_main_cards')
all_inn_data

,inn_smi_publications,inn_company_main_cards
0,9704082895,9704082895
1,7730569353,7730569353
2,7703400140,7703400140
3,8603118208,8603118208
4,7710068052,7710068052
...,...,...
558,7708503727,7708503727
559,7706107510,7706107510
560,7708004767,7708004767
561,7736050003,7736050003


In [76]:
data = data.merge(all_inn_data, how='left', left_on='inn_file', right_on='inn_smi_publications')
data

,company_file,inn_file,industry_file,inn_smi_publications,inn_company_main_cards
0,"«КТЗ», промышленный оператор",9704082895,оптовая торговля,9704082895,9704082895
1,«Автомир-Трейд»,7730569353,розничная торговля,7730569353,7730569353
2,«Джи Пи Си Рус»,7703400140,платежный сервис,7703400140,7703400140
3,«Самотлорнефтепромхим»,8603118208,сервисные компании,8603118208,8603118208
4,«Мосметрострой»,7710068052,"инжиниринг, промышленно-инфраструктурное строи...",7710068052,7710068052
...,...,...,...,...,...
583,X5 Group,7733571872,розничная торговля,7733571872,7733571872
584,РЖД,7708503727,транспорт и логистика,7708503727,7708503727
585,"«Роснефть», нефтяная компания",7706107510,нефтяная и нефтегазовая промышленность,7706107510,7706107510
586,«ЛУКОЙЛ»,7708004767,нефтяная и нефтегазовая промышленность,7708004767,7708004767


In [77]:
data_clean = data.dropna()
data_clean

,company_file,inn_file,industry_file,inn_smi_publications,inn_company_main_cards
0,"«КТЗ», промышленный оператор",9704082895,оптовая торговля,9704082895,9704082895
1,«Автомир-Трейд»,7730569353,розничная торговля,7730569353,7730569353
2,«Джи Пи Си Рус»,7703400140,платежный сервис,7703400140,7703400140
3,«Самотлорнефтепромхим»,8603118208,сервисные компании,8603118208,8603118208
4,«Мосметрострой»,7710068052,"инжиниринг, промышленно-инфраструктурное строи...",7710068052,7710068052
...,...,...,...,...,...
583,X5 Group,7733571872,розничная торговля,7733571872,7733571872
584,РЖД,7708503727,транспорт и логистика,7708503727,7708503727
585,"«Роснефть», нефтяная компания",7706107510,нефтяная и нефтегазовая промышленность,7706107510,7706107510
586,«ЛУКОЙЛ»,7708004767,нефтяная и нефтегазовая промышленность,7708004767,7708004767


In [78]:
smi_card_path = path+'company_smi_publications/'

def organizations_checker(df, row_number):
  company_inn = str(df['inn_file'].iloc[row_number])
  print(company_inn)

  with open(smi_card_path+company_inn+'.html', 'r') as f:
    contents = f.read()

    soup = BeautifulSoup(contents, 'lxml')
    dom = etree.HTML(str(soup))

  side_panels = soup.find('div', {"class": "sidebar-filters"}).find_all('div', {'class': 'sidebar-filters__section sidebar-filters__section_no-delimiter'})
  ammt_side_panels = len(side_panels)

  org_panel_id = -1
  for panel in range(ammt_side_panels):
    name = dom.xpath(f'/html/body/div/div[1]/div[1]/div[1]/div[2]/div/div/div/div/div[3]/div[2]/div[2]/div[2]/div[1]/div/div[{panel+1}]/div/div/div/button/div/text()')
    if name:
      if name[0].upper() == 'ОРГАНИЗАЦИИ':
        org_panel_id = panel+1

  flag = -1
  if 'еще' in str(side_panels[org_panel_id-1]).lower():
    flag = 0
  else:
    flag = 1

  f.close()

  return flag

In [79]:
organizations_checker_marker = []

for row in range(data_clean.shape[0]):
  flg = organizations_checker(data_clean, row)
  organizations_checker_marker.append(flg)

9704082895
7730569353
7703400140
8603118208
7710068052
1655049111
7709340842
9718221490
7719022542
7714775020
5042060699
7704758153
5029104266
7838418751
2315995868
5403145086
5047172389
7707073366
1654035998
4826022365
7705639173
1627005779
7705937571
7727696432
1655189422
6632004667
7710698978
7701655865
6663003800
7732121736
5046001101
4623004836
7717127211
5047105054
2922008546
4633037968
3814007314
3305051742
7703295463
4909105495
7810022460
5609087920
6229024796
9705118142
7707205911
6732075238
7709583813
3811125221
7730107662
7701004101
5001048893
4725001168
9715239467
4203001913
7811657720
7704160892
7709976356
0270007455
7718538045
7705189397
7705532790
7830001028
4216001565
7706236515
7703106050
7017286512
1001228754
7428007317
7702192017
7705005321
5827901723
4252012308
7710761161
7731530768
7729507377
7716641537
7705058323
8904075533
7702352454
5030041780
7814581879
3812100526
7705040943
3328430578
7327067461
7839326623
7719555477
7703248907
1303066789
7810349385
7706739893

In [80]:
data_clean['open_orgs_checker'] = organizations_checker_marker
data_clean

<ipython-input-80-1b28d6974358>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['open_orgs_checker'] = organizations_checker_marker


,company_file,inn_file,industry_file,inn_smi_publications,inn_company_main_cards,open_orgs_checker
0,"«КТЗ», промышленный оператор",9704082895,оптовая торговля,9704082895,9704082895,1
1,«Автомир-Трейд»,7730569353,розничная торговля,7730569353,7730569353,1
2,«Джи Пи Си Рус»,7703400140,платежный сервис,7703400140,7703400140,1
3,«Самотлорнефтепромхим»,8603118208,сервисные компании,8603118208,8603118208,1
4,«Мосметрострой»,7710068052,"инжиниринг, промышленно-инфраструктурное строи...",7710068052,7710068052,1
...,...,...,...,...,...,...
583,X5 Group,7733571872,розничная торговля,7733571872,7733571872,1
584,РЖД,7708503727,транспорт и логистика,7708503727,7708503727,0
585,"«Роснефть», нефтяная компания",7706107510,нефтяная и нефтегазовая промышленность,7706107510,7706107510,1
586,«ЛУКОЙЛ»,7708004767,нефтяная и нефтегазовая промышленность,7708004767,7708004767,0


In [81]:
data_for_info = data.merge(data_clean[['inn_file', 'open_orgs_checker']], how='left', on='inn_file')
data_for_info = data_for_info[data_for_info['open_orgs_checker'] == 1]

In [82]:
data_for_info

,company_file,inn_file,industry_file,inn_smi_publications,inn_company_main_cards,open_orgs_checker
0,"«КТЗ», промышленный оператор",9704082895,оптовая торговля,9704082895,9704082895,1.0
1,«Автомир-Трейд»,7730569353,розничная торговля,7730569353,7730569353,1.0
2,«Джи Пи Си Рус»,7703400140,платежный сервис,7703400140,7703400140,1.0
3,«Самотлорнефтепромхим»,8603118208,сервисные компании,8603118208,8603118208,1.0
4,«Мосметрострой»,7710068052,"инжиниринг, промышленно-инфраструктурное строи...",7710068052,7710068052,1.0
...,...,...,...,...,...,...
571,"«Мегаполис», группа компаний",5003052454,оптовая торговля,5003052454,5003052454,1.0
573,«Транснефть» (ПАО),7706061801,транспорт и логистика,7706061801,7706061801,1.0
576,ПАО «Новолипецкий металлургический комбинат» (...,4823006703,черная металлургия,4823006703,4823006703,1.0
583,X5 Group,7733571872,розничная торговля,7733571872,7733571872,1.0


# Основная карточка компании

In [95]:
company_main_cards_path = path+'company_smi_publications/'

lst_of_companies = (data_for_info['inn_file'] + '.html').to_list()

In [96]:
def smi_inn_crawler(file_name):
  with open(company_main_cards_path+file_name, 'r') as f:

    contents = f.read()
    soup = BeautifulSoup(contents, 'lxml')

  dom = etree.HTML(str(soup))
  company_main_info = {}
  for i in range(1,5):
    info_type = dom.xpath(f'/html/body/div/div[1]/div[1]/div[1]/div[1]/div/div/div[2]/div/span[{i}]/span/span/span/span/text()')[0][:-1]
    info = dom.xpath(f'/html/body/div/div[1]/div[1]/div[1]/div[1]/div/div/div[2]/div/span[{i}]/span/span/span/span/span/span/span/text()')[0]
    company_main_info[info_type] = info

  return company_main_info

In [97]:
smi_info_checker = []

for comp in lst_of_companies:
  print(comp)
  smi_info_checker.append(smi_inn_crawler(comp))

9704082895.html
7730569353.html
7703400140.html
8603118208.html
7710068052.html
1655049111.html
7709340842.html
9718221490.html
7719022542.html
7714775020.html
5042060699.html
7704758153.html
5029104266.html
7838418751.html
2315995868.html
5403145086.html
5047172389.html
1654035998.html
4826022365.html
7705639173.html
7727696432.html
1655189422.html
6632004667.html
7710698978.html
7701655865.html
6663003800.html
7732121736.html
5046001101.html
4623004836.html
7717127211.html
5047105054.html
2922008546.html
4633037968.html
3814007314.html
3305051742.html
7703295463.html
7810022460.html
5609087920.html
6229024796.html
9705118142.html
7707205911.html
6732075238.html
7709583813.html
3811125221.html
7730107662.html
7701004101.html
5001048893.html
4725001168.html
9715239467.html
4203001913.html
7811657720.html
7709976356.html
7718538045.html
7705189397.html
7705532790.html
7830001028.html
4216001565.html
7706236515.html
7703106050.html
7017286512.html
1001228754.html
7428007317.html
77021920

In [98]:
res = pd.DataFrame(smi_info_checker)
res

,ОГРН,ИНН,КПП,ОКПО
0,1217700375991,9704082895,210001001,51974376
1,1077761079263,7730569353,771801001,83121511
2,1157746967510,7703400140,774301001,51166895
3,1048600521408,8603118208,860301001,74733014
4,1027739002378,7710068052,770701001,01387120
...,...,...,...,...
474,1045000923967,5003052454,500301001,75219395
475,1027700049486,7706061801,770301001,00044463
476,1024800823123,4823006703,482301001,05757665
477,1067746744955,7733571872,773301001,96042871


In [99]:
res = pd.DataFrame(smi_info_checker)
c = pd.merge(res, data_for_info, right_on='inn_file', left_on='ИНН', how='right')
# test = test.dropna()

In [101]:
test.isna().sum()

ОГРН                      0
ИНН                       0
КПП                       0
ОКПО                      0
company_file              0
inn_file                  0
industry_file             0
inn_smi_publications      0
inn_company_main_cards    0
open_orgs_checker         0
dtype: int64

# Публикации в СМИ

In [92]:
company_smi_publications_path = path+'company_main_cards/'

lst_of_companies = (data_for_info['inn_file'] + '.html').to_list()

In [93]:
def smi_inn_crawler(file_name):
  with open(company_smi_publications_path+file_name, 'r') as f:

    contents = f.read()
    soup = BeautifulSoup(contents, 'lxml')

  dom = etree.HTML(str(soup))
  company_main_info = {}
  for i in range(1,5):
    info_type = dom.xpath(f'/html/body/div/div[1]/div[1]/div[1]/div[1]/div/div/div[2]/div/span[{i}]/span/span/span/span/text()')[0][:-1]
    info = dom.xpath(f'/html/body/div/div[1]/div[1]/div[1]/div[1]/div/div/div[2]/div/span[{i}]/span/span/span/span/span/span/span/text()')[0]
    company_main_info[info_type] = info

  return company_main_info

In [ ]:
smi_info_checker = []

for comp in lst_of_companies:
  print(comp)
  smi_info_checker.append(smi_inn_crawler(comp))

In [86]:
res = pd.DataFrame(smi_info_checker)
res

,ОГРН,ИНН,КПП,ОКПО
0,1217700375991,9704082895,210001001,51974376
1,1077761079263,7730569353,771801001,83121511
2,1157746967510,7703400140,774301001,51166895
3,1048600521408,8603118208,860301001,74733014
4,1027739002378,7710068052,770701001,01387120
...,...,...,...,...
474,1045000923967,5003052454,500301001,75219395
475,1027700049486,7706061801,770301001,00044463
476,1024800823123,4823006703,482301001,05757665
477,1067746744955,7733571872,773301001,96042871


In [87]:
res = pd.DataFrame(smi_info_checker)
test = pd.merge(res, data_for_info, right_on='inn_file', left_on='ИНН', how='right')
# test = test.dropna()

In [88]:
test[test['ОГРН'].isna()]

,ОГРН,ИНН,КПП,ОКПО,company_file,inn_file,industry_file,inn_smi_publications,inn_company_main_cards,open_orgs_checker
